# **Resultados da avalição do sistema de exercícios Machine Teaching**
# 2019/2, PLE, 2020, 2021

#### Importando bibliotecas e dados

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, Markdown
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
#path = 'drive/MyDrive/computação_e_educação/avaliação/resultados/'
path = 'drive/MyDrive/UFRJ/Pesquisa/Educação/Computação e Educação/avaliação/resultados/'

Mounted at /content/drive/


In [2]:
#@markdown Importação dos dados
alunos_ple = "ple_alunos.csv"
alunos_2019_2 = "2019_2_alunos.csv"
alunos_2020_1 = "2020_1_alunos.csv"
alunos_2020_2 = "2020_2_alunos.csv"
alunos_2021_1 = "2021_1_alunos.csv"
alunos_2022_1 = "2021_2_alunos.csv"

professores_2020_1 = "2020_1_professores.csv"
professores_2020_2 = "2020_2_professores.csv"
professores_2021_1 = "2021_1_professores.csv"
professores_2021_2 = "2021_2_professores.csv"

alunos_19_2 = pd.read_csv(path+alunos_2019_2)
alunos_ple = pd.read_csv(path+alunos_ple)
alunos_20_1 = pd.read_csv(path+alunos_2020_1)
alunos_20_2 = pd.read_csv(path+alunos_2020_2)
alunos_21_1 = pd.read_csv(path+alunos_2021_1)
alunos_21_2 = pd.read_csv(path+alunos_2022_1)
professores_20_1 = pd.read_csv(path+professores_2020_1)
professores_20_2 = pd.read_csv(path+professores_2020_2)
professores_21_1 = pd.read_csv(path+professores_2021_1)
professores_21_2 = pd.read_csv(path+professores_2021_2)

# Análise de dados

### Funções auxiliares

In [3]:
def get_valid_cols(data):
  valid_cols = []
  for col in data.columns:
    if (data[col].dtype == np.dtype('int64')) or (data[col].dtype == np.dtype('float64')):
      valid_cols.append(col)
  return valid_cols
 
def boxplot(data):
  valid_cols = get_valid_cols(data)
  fig = go.Figure()
  
  updatemenu = []
  buttons = []
  for idx, col in enumerate(valid_cols):
      fig.add_trace(go.Box(x=data['semester'], y=data[col], 
                           visible=False))
      
      visible = [False]*len(valid_cols)
      visible[idx] = True
      buttons.append(dict(method='restyle', label=col,
                          args=[{"visible": visible},
                                {"title": col,
                                "annotations": []}]))

  
  updatemenu = []
  menu = dict()
  updatemenu.append(menu)
  updatemenu[0] = dict(buttons=buttons, direction='down', showactive=True,
                  pad={"r": 10, "t": 10}, x=0, xanchor="left", y=1.2, yanchor="top")
  fig.update_layout(showlegend=False, 
                    updatemenus=updatemenu,
                     font=dict(
                        size=14,),
                         yaxis = dict(
       # tickmode = 'array',
      #  tickvals = [1,2,3,4,5,6],
    ))
  fig.show()

def bar(data, cols=['semester'], legend=False, percent=False):
  data_hist = data.value_counts(subset=cols)
  data_hist = data_hist.reset_index()
  data_hist.columns = cols + ['count']
  
  if percent:
    data_hist = data_hist.apply(calculate_percentage, 
                                args=[data_hist, cols[0]], axis=1)
    count_col = 'percentage'
  else:
    count_col = 'count'

  if len(cols) == 1:
    fig = px.bar(data_hist, x=cols[0], y=count_col, text_auto=True)
  elif len(cols) == 2:
    fig = px.bar(data_hist, x=cols[0], y=count_col, color=cols[1], text=cols[1])
  else:
    raise NotImplementedError("Mais de duas colunas para plotar")
    
  fig.update_layout(xaxis={'categoryorder':'category ascending'},
                    showlegend=legend, barmode='relative')
  fig.show()

def add_semester(dfs, titles):
  for idx, df in enumerate(dfs):
    df["semester"] = titles[idx]
  return dfs

def concat_and_sort(dfs):
  data = pd.concat(dfs)
  cols = data.columns
  cols = cols.sort_values()
  return data[cols]

def get_columns_per_semester(data):
  cols_dict = {}
  for col in data.columns:
    if (data[col].dtype == np.dtype('int64')) or (data[col].dtype == np.dtype('float64')):
      d = data[['semester', col]]
      d = d.dropna()
      semesters = d['semester'].unique()
      cols_dict[col] = []
      for s in titles:
        if s in semesters:
          cols_dict[col].append(1)
        else:
          cols_dict[col].append(0)
    else:
      pass
      #print(f"{col} não é numérica")
      #print(data[col].dtype)

  cols_df = pd.DataFrame.from_dict(cols_dict, orient='index', columns=titles)
  cols_df.to_csv("cols_per_semester.csv", sep=";", encoding='iso-8859-1')
  return cols_df

def merge_cols(data, col1, col2):
  data.loc[:, col1] = data.loc[:,[col1, col2]].sum(axis=1, min_count=1)
  return data

def calculate_percentage(row, data, col):
  query = data[data[col] == row[col]]
  total_count = query['count'].sum()
  row['percentage'] = row['count'] / total_count
  return row

### **Alunos**

#### Total de respostas por ano

##### Preparando plot

In [4]:
titles = ["2019/2", "2020/0 (PLE)", "2020/1", "2020/2", "2021/1", "2021/2"]
dfs = [alunos_19_2, alunos_ple, alunos_20_1,alunos_20_2, alunos_21_1, alunos_21_2]

# Adiciona a informação de semestre
dfs = add_semester(dfs, titles)
# Concatena dfs e ordena colunas
data = concat_and_sort(dfs)

##### Total de respostas por ano

In [5]:
bar(data)

#### Removendo colunas não mais utilizadas e juntando colunas similares

In [6]:
cols_df = get_columns_per_semester(data)
cols_to_remove = cols_df[(cols_df['2020/0 (PLE)'] == 1) & (cols_df['2020/1'] == 0) & (cols_df['2021/2'] == 0)]
cols_to_remove.index.tolist()

['Quando a resposta estava errada, a indicação de quais entradas estavam erradas me ajudava a resolver.',
 'Quando a resposta estava errada, o erro apontado no Console (parte preta) me ajudava a resolver.']

In [7]:
data = data.drop(columns = cols_to_remove.index.tolist())

In [8]:
merge_col1 = "Eu estudei ou pretendo estudar pelo sistema para fazer a prova"
merge_col2 = "Eu estudei pelo sistema para fazer as provas"
data = merge_cols(data, merge_col1, merge_col2)

merge_col1 = "Eu tive facilidade em encontrar os exercícios passados para estudar"
merge_col2 = "Eu tive facilidade em encontrar os exercícios das listas das semanas passadas para estudar"
data = merge_cols(data, merge_col1, merge_col2)

In [9]:
data[merge_col1]

0     5.0
1     4.0
2     3.0
3     3.0
4     4.0
     ... 
78    5.0
79    2.0
80    5.0
81    4.0
82    5.0
Name: Eu tive facilidade em encontrar os exercícios passados para estudar, Length: 721, dtype: float64

#### Perguntas 

In [10]:
boxplot(data)

#### Limpando os dados

In [11]:
col = 'Ao responder uma questão, onde você resolvia ela?'

map_dict = {
    "Escrevia o código somente no sistema": "Sistema",
    "Tentava primeiro no sistema e se desse errado, tentava no IDLE": "Sistema e IDLE",
    "Escrevia primeiro no IDLE e depois passavapara o sistema": "IDLE e Sistema",
    "Escrevia primeiro no IDLE e depois passava para o sistema": "IDLE e Sistema",
}

replace_dict = map_dict.items()
replace_list = list(zip(*replace_dict))
data[col] = data[col].replace(replace_list[0], replace_list[1])

#### Plots

In [12]:
col = 'Ao responder uma questão, onde você resolvia ela?'
bar(data, cols=["semester", col])
bar(data, cols=["semester", col], percent=True)

In [13]:
col = 'Com que frequência você utilizou o dashboard de acompanhamento em “Resultados”?'
bar(data, cols=["semester", col])
bar(data, cols=["semester", col], percent=True)

In [14]:
#@title  { run: "auto" }
#@markdown Sugestões de melhorias
mostrar = 500 #@param {type:"slider", min:0, max:500, step:1}
sugestoes = []
for item in alunos_20_1.iloc[:,-1].dropna():
  sugestoes.append(item)
for item in alunos_20_2.iloc[:,-1].dropna():
  sugestoes.append(item)
for item in alunos_21_1.iloc[:,-1].dropna():
  sugestoes.append(item)
for item in sugestoes[:mostrar]:
  display(Markdown(f'**{n+1}.**  {item}'))

NameError: ignored

In [ ]:
len(sugestoes)

In [ ]:
keep_erro = [4, 9, 11, 13, 14, 16, 22, 26, 28, 37, 38, 39, 41, 44, 48, 49, 51, 52, 53]
len(keep)
tutor = 4


In [ ]:
counter = 0
keep = []
for item in sugestoes:
  if 'tutor' in item:      
    print(counter)
    print(item)
    print()
    counter += 1

### **Professores**

In [ ]:
boxplot(professores_20_1, professores_20_1, professores_20_1,professores_20_2, professores_21_1)

In [ ]:
bar(professores_20_1,professores_20_2, professores_21_1, 9)

In [ ]:
#@title  { run: "auto" }
#@markdown Opiniões sobre casos de teste ocultos

respostas = list(professores_20_1.iloc[:,10])+list(professores_20_2.iloc[:,10])
for n, item in enumerate(respostas):
  display(Markdown(f'**{n+1}.**  {item}'))

In [ ]:
#@title  { run: "auto" }
#@markdown Sugestões de melhorias

mostrar = 100 #@param {type:"slider", min:0, max:100, step:1}
sugestoes = []
for item in professores_20_1.iloc[:,-1].dropna():
  sugestoes.append(item)
for item in professores_20_2.iloc[:,-1].dropna():
  sugestoes.append(item)
for item in professores_21_1.iloc[:,-1].dropna():
  sugestoes.append(item)
for n, item in enumerate(sugestoes[:mostrar]):
  display(Markdown(f'**{n+1}.**  {item}'))

In [ ]:
len(sugestoes)